## Verifique si el valor de stars en business.csv es válido y consistente. ¿Qué porcentaje del campo es válido?

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 48 kB/s 
     |████████████████████████████████| 199 kB 40.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=21ec21396d17efa96872aebad3b0b8a5d214e3646396dd1c827fb7ed82e9ed45
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://archive.

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [ ]:
df_prueba2 = sqlContext.read.parquet('/content/drive/MyDrive/ODD tp 1 dataset/pablo2.parquet', header=True, inferSchema=True)
rdd_review = df_prueba2.rdd

In [ ]:
df_prueba3 = sqlContext.read.parquet('/content/drive/MyDrive/ODD tp 1 dataset/business2.parquet', header=True, inferSchema=True)
rdd_bis= df_prueba3.rdd

In [ ]:
rdd_bis.take(2)

[Row(business_id='Pns2l4eNsfO8kk83dixA6A', name='Abby Rappoport, LAC, CMQ', address='1616 Chapala St, Ste 2', city='Santa Barbara', state='CA', postal_code='93101', latitude=34.4266787, longitude=-119.7111968, stars=5.0, review_count=0.0, is_open=0, attributes="{'ByAppointmentOnly': 'True'}", categories='Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists', hours=None),
 Row(business_id='mpf3x-BjTdTEA3yCZrAYPw', name='The UPS Store', address='87 Grasso Plaza Shopping Center', city='Affton', state='MO', postal_code='63123', latitude=38.551126, longitude=-90.335695, stars=3.0, review_count=8.0, is_open=1, attributes="{'BusinessAcceptsCreditCards': 'True'}", categories='Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services', hours="{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', 'Wednesday': '8:0-18:30', 'Thursday': '8:0-18:30', 'Friday': '8:0-18:30', 'Saturday': '8:0-14:0'}")]

In [ ]:
rdd_bis = rdd_bis.map(lambda x: (x.business_id, x.stars))

In [ ]:
rdd_bis.take(2)

[('Pns2l4eNsfO8kk83dixA6A', 5.0), ('mpf3x-BjTdTEA3yCZrAYPw', 3.0)]

In [ ]:
rdd_bis.count()

150346

In [ ]:
rdd_review.take(5)

[Row(business_id='XQfwVwDr-v0ZS3_CbbE5Xw', stars=3.0),
 Row(business_id='e4Vwtrqf-wpJfwesgvdgxQ', stars=4.0),
 Row(business_id='cPepkJeRMtHapc_b2Oe_dw', stars=4.0),
 Row(business_id='bMratNjTG5ZFEA6hVyr-xQ', stars=5.0),
 Row(business_id='bbEXAEFr4RYHLlZ-HFssTA', stars=5.0)]

In [ ]:
rdd_review = rdd_review.map(lambda x: (x.business_id, x.stars))

In [ ]:
from builtins import round

# Calculo el promedio por business en el review.csv para despues joinear con el business.csv y chequear si los valores son validos y consistentes

In [ ]:
rdd_review = rdd_review \
    .mapValues(lambda v: (v, 1)) \
    .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1])) \
    .mapValues(lambda v: round(v[0]/v[1], 1))

In [ ]:
rdd_review.take(5)

[('XQfwVwDr-v0ZS3_CbbE5Xw', 3.1),
 ('bMratNjTG5ZFEA6hVyr-xQ', 3.9),
 ('bbEXAEFr4RYHLlZ-HFssTA', 3.9),
 ('A2q7d-CBM2-81tVkmS4JMw', 3.3),
 ('qO9dNNIvNbCBd8ZgjxMxgQ', 4.7)]

In [ ]:
rdd_joineado = rdd_review.join(rdd_bis)

# Aca ya tengo las tuplas para chequear la validez de los valores

In [ ]:
rdd_joineado.take(5)

[('bMratNjTG5ZFEA6hVyr-xQ', (3.9, 4.0)),
 ('A2q7d-CBM2-81tVkmS4JMw', (3.3, 3.5)),
 ('cg4JFJcCxRTTMmcg9O9KtA', (3.3, 3.5)),
 ('S2Ho8yLxhKAa26pBAm6rxA', (3.8, 4.0)),
 ('1QVB0_-piu0GXes87BXeGw', (4.6, 4.5))]

# Cuento los valores que son iguales y los sumo, luego divido y redondeo por la cantidad de valores totales (los cuales también cuento) para obtener el porcentaje de valores que son iguales, es decir, el porcentaje de campo válido.

In [ ]:
porcentaje_campo_valido = rdd_joineado.flatMap(lambda x:  [(x[0], 1 if (x[1][0] == x[1][1]) else 0)]) \
                          .map(lambda x: (x[1], 1)) \
                          .reduceByKey(lambda a,b: a+b) \
                          .reduce(lambda a,b: round(((b[1]*100) / (a[1]+b[1])),2))

In [ ]:
porcentaje_campo_valido 

20.53